In [3]:
from pyperplan import _parse, _ground
import numpy as np

In [4]:
problem = _parse(domain_file='testdomain.pddl',problem_file='testinstance.pddl')
task = _ground(problem)

## Naive embedding

- Naive binary embedding for a state space with N fluents is a N-dimensional binary vector that has a 1 for each true fact
- Naive continuous embedding is just the binary embedding times an NxM trainable embedding matrix
- Naive because we aren't using any shared feature representations for similar predicates/facts

In [32]:
def create_naive_embedding_fn(facts):
  """NOTE: Probably not best implementation atm; should be faster to initialize np.zeros, then 
  get indices of the state with a hash/dict, and set those indices to 1"""
  facts = list(facts)
  def naive_embedding(state):
    return np.array([(fact in state) for fact in facts], dtype=np.float32)
  return naive_embedding

In [30]:
naive_emb = create_naive_embedding_fn(task.facts)

In [31]:
# So the binary embedding of the initial state is:
naive_emb(task.initial_state)

array([1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1.],
      dtype=float32)

In [28]:
# We can now get an N-dimensional continuous embedding using a trainable embedding matrix
# This is equivalent to having an N-dimensional embedding for each "fact/fluent", and then adding them together.
N = 16
W_emb = np.random.normal(0., 1e-2, (len(task.facts), N))

# So now the initial embedding of the initial state is:
np.expand_dims(naive_emb(task.initial_state), 0).dot(W_emb)

array([[ 0.01330587,  0.00131232, -0.02270757,  0.01784888,  0.05183817,
        -0.01747334, -0.04812877, -0.03406031, -0.02066482,  0.00268202,
        -0.01011077,  0.0104214 ,  0.05642818, -0.01336079,  0.04047353,
        -0.0518132 ]])

## Smarter embedding

- this is non-trivial... we should discuss